In [52]:
# 파이썬 기반의 동적 수집 실습(3) - 동적 수집 수행평가
from selenium import webdriver
import re
import time

# 랜더링
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)
driver.get('http://gs25.gsretail.com/gscvs/ko/products/event-goods')
time.sleep(3)

# 2+1 클릭
tobt = driver.find_element_by_css_selector("#TWO_TO_ONE")
tobt.click()
goodsname = []
goodsprice = []
page = 1

# 동적으로 데이터 수집
while True:
    for i in range(1,9):
        name = driver.find_elements_by_css_selector('#contents div:nth-child(5) > ul > li:nth-child('+str(i)+') > div > p.tit')
        price = driver.find_elements_by_css_selector('#contents div:nth-child(5) > ul > li:nth-child('+str(i)+') > div > p.price > span')

        for nDom in name:
            goodsname.append(nDom.text)

        for pDom in price:
            p = re.sub("[,]|['원']", "", pDom.text)
            goodsprice.append(p)
    
    
    nextbt = driver.find_element_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > div > a.next')
    nextbt.click()
    time.sleep(1)
    page = page + 1
    check = driver.find_elements_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > div > span > a')
    n = (page-1) % 10
    try: 
        int(check[n].text)
    except:
        print((page-1), "페이지까지 크롤링 종료")
        break

# csv 파일 저장
f = open("output/gs25_twotoone.csv", "w")
f.write('상품명'+','+'가격'+'\n')
for i in range(len(goodsname)):
    f.write(goodsname[i]+ ',' + goodsprice[i] + '\n')    
f.close()
driver.quit()

103 페이지까지 크롤링 종료


### 메모

#### 마지막 페이지 조건 시 list의 범위가 초과되어 오류가 남 -> try except를 이용하여 루프 탈출 조건 설정함
#### csv에서는 데이터의 열 구분을 , 로 행 구분을 \n로 함!

<br>
새로운 방법: DataFrame 활용하여 csv 저장!

In [ ]:
# 파이썬 기반의 동적 수집 실습(3) - 동적 수집 수행평가
from selenium import webdriver
import re
import time
import pandas as pd

# 랜더링
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)
driver.get('http://gs25.gsretail.com/gscvs/ko/products/event-goods')
time.sleep(3)

# 2+1 클릭
tobt = driver.find_element_by_css_selector("#TWO_TO_ONE")
tobt.click()
goodsname = []
goodsprice = []
page = 1

# 동적으로 데이터 수집
while True:
    for i in range(1,9):
        name = driver.find_elements_by_css_selector('#contents div:nth-child(5) > ul > li:nth-child('+str(i)+') > div > p.tit')
        price = driver.find_elements_by_css_selector('#contents div:nth-child(5) > ul > li:nth-child('+str(i)+') > div > p.price > span')

        for nDom in name:
            goodsname.append(nDom.text)

        for pDom in price:
            p = re.sub("[,]|['원']", "", pDom.text)
            goodsprice.append(p)
    
    
    nextbt = driver.find_element_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > div > a.next')
    nextbt.click()
    time.sleep(1)
    page = page + 1
    check = driver.find_elements_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > div > span > a')
    n = (page-1) % 10
    try: 
        int(check[n].text)
    except:
        print((page-1), "페이지까지 크롤링 종료")
        break

# df이용하여 csv 파일 저장
df7 = (pd.DataFrame([goodsname, goodsprice])).T
df7.columns = ['상품명', '가격']
display(df7)
df7.to_csv("output/gs25_twotoone_new.csv", encoding="utf-8-sig")
driver.quit()